In [1]:
#imports
import pyvisa
import numpy as np
import time
import ipywidgets as wg
from IPython.display import display

In [2]:
#Functions
def load_primary_waveform(Sample_Rate,VPP,Offset):
    rigol.write(f":SOUR1:APPL:ARB {Sample_Rate},{VPP},{Offset}")
    rigol.write(send_wave_primary)
    rigol.write(":SOUR1:BURSt:STATe ON")
    rigol.write(":SOUR1:BURSt:MODE TRIGgered")
    rigol.write(":SOUR1:BURSt:NCYCles 1")
    rigol.write(":SOUR1:BURSt:TRIGger:SOURce MANual")
    rigol.write(":SOUR1:BURSt:IDLE BOTTOM")

def load_return_waveform(Sample_Rate,VPP,Offset):
    rigol.write(f":SOUR1:APPL:ARB {Sample_Rate},{VPP},{Offset}")
    rigol.write(send_wave_return)
    rigol.write(":SOUR1:BURSt:STATe ON")
    rigol.write(":SOUR1:BURSt:MODE TRIGgered")
    rigol.write(":SOUR1:BURSt:NCYCles 1")
    rigol.write(":SOUR1:BURSt:TRIGger:SOURce MANual")
    rigol.write(":SOUR1:BURSt:IDLE TOP")

def trigger_descending_wave(click):
    if rigol.query(':SOUR1:BURST:IDLE?') != "TOP":
        raise ValueError("Wave cannot descend as voltage is not at maximum")
        return
    load_primary_waveform(5000,int(max_voltage.value),(int(max_voltage.value)/2))
    time.sleep(3)
    rigol.write(":SOUR1:BURST:TRIG")

def trigger_ascending_wave(click):
    if rigol.query(':SOUR1:BURST:IDLE?') != "BOTTOM":
        raise ValueError("Wave cannot ascend as voltage is not at minimum")
        return
    load_return_waveform(5000,int(max_voltage.value),0)
    time.sleep(3)
    rigol.write(":SOUR1:BURST:TRIG")
    
def initialize_bottom(click):
    load_return_waveform(5000,int(max_voltage.value),0)

In [3]:
#User inputs
frequency = wg.Text(value = "Frequency")
display(frequency)
time_length = wg.Text(value = "Time Length of Waveform")
display(time_length)
decimals= wg.Text(value = "Number of Decimals")
display(decimals)
amplitude = wg.Text(value = "Ampltiude")
display(amplitude)
num_points = wg.Text(value = "Number of Points in Waveform")
display(num_points)
max_voltage = wg.Text(value = "Maximum waveform voltage")
display(max_voltage)

Text(value='Frequency')

Text(value='Time Length of Waveform')

Text(value='Number of Decimals')

Text(value='Ampltiude')

Text(value='Number of Points in Waveform')

Text(value='Maximum waveform voltage')

In [4]:
#establish connection with instrument
rm = pyvisa.ResourceManager()
print(rm.list_resources())
rigol = rm.open_resource('USB0::0x1AB1::0x0642::DG1ZA221601025::INSTR')
print(rigol.query('*IDN?'))
rigol.read_termination = '\n'
rigol.write_termination = '\n'

('USB0::0x1AB1::0x0642::DG1ZA221601025::INSTR',)
Rigol Technologies,DG1022Z,DG1ZA221601025,03.01.12  



In [5]:
# inputs for primary waveform
# frequency = 250           # frequency(Hz)
# time_length = 10           # total time of the waveform (s)
# decimals = 3              # number of decimal places to keep in the waveform
# amplitude = 1            # amplitude (V)
# num_points = 201          # number of points to put in the waveform

In [6]:
# inputs for return waveform
# frequency_return = 250         # frequency(Hz)
# time_length = 10           # total time of the waveform (s)
# decimals = 3              # number of decimal places to keep in the waveform
# amplitude = 1            # amplitude (V)
# num_points = 201          # number of points to put in the waveform

In [7]:
# create time vector in seconds for primary
# t_primary = np.array(list(range(num_points)))
# t_primary = t_primary/((num_points-1)/time_length)

t = np.array(list(range(int(num_points.value))))
t = t/((int(num_points.value)-1)/int(time_length.value))

# create time vector in seconds for return
# t_return = np.array(list(range(num_points)))
# t_return = t_return/((num_points-1)/time_length)

# create voltage vector v(t) for primary
v_primary = -1/(1+np.exp(5-t))   # change this equation, ensuring v does not exceed amplitude
v_primary = v_primary/int(amplitude.value)
v_primary = np.round(v_primary, int(decimals.value))

# create voltage vector v(t) for return
v_return = 1/(1+np.exp(5-t))   # change this equation, ensuring v does not exceed amplitude
v_return = v_return/int(amplitude.value)
v_return = np.round(v_return, int(decimals.value))

# check v for exceeding amplitude in either waveform
if any(v_primary>1) or any(v_primary<-1):
    print(v_primary)
    raise ValueError("Voltage exceeds allowed amplitude")

if any(v_return>1) or any(v_return<-1):
    print(v_return)
    raise ValueError("Voltage exceeds allowed amplitude")

ValueError: invalid literal for int() with base 10: 'Number of Points in Waveform'

In [ ]:
# creating commands:

# set frequency, amplitude command for primary
freq_set_primary = f":SOUR1:APPL:ARB {int(frequency.value)},{int(amplitude.value)}"

# set frequency, amplitude command for return
freq_set_return = f":SOUR1:APPL:ARB {int(frequency.value)},{int(amplitude.value)}"

# send waveform command for primary
v_primary = v_primary.astype(str)
pts_primary = ",".join(v_primary)
send_wave_primary = f":SOUR1:DATA VOLATILE,{pts_primary}"

# send waveform command for return
v_return = v_return.astype(str)
pts_return = ",".join(v_return)
send_wave_return = f":SOUR1:DATA VOLATILE,{pts_return}"

# turn on output
on = ":OUTP1 ON"

print(freq_set_primary, '\n\n', send_wave_primary, "\n\n", on)
print(freq_set_return, '\n\n', send_wave_return, "\n\n", on)

In [ ]:
#load in the primary waveform
# trigger_descending_wave(5000,10,5)

In [ ]:
#load in the return waveform
# trigger_ascending_wave(5000,10,0)

In [ ]:
# rigol.write(":SOUR1:BURST OFF")

In [ ]:
# rigol.write("OUTP1 ON")

In [ ]:
# initialize_bottom(5000,10,0)

In [ ]:
#UI Buttons
initialize_waveform = wg.Button(
    description='Initialize Waveform',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='This will initialize the waveform to zero volts in preparation for an ascending waveform',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
display(initialize_waveform)
initialize_waveform.on_click(initialize_bottom)

ascending_waveform = wg.Button(
    description='Ascending Waveform',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='This will trigger the ascending waveform',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
display(ascending_waveform)
ascending_waveform.on_click(trigger_ascending_wave)

descending_waveform = wg.Button(
    description='Descending Waveform',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='This will trigger the descending waveform',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
display(descending_waveform)
descending_waveform.on_click(trigger_descending_wave)


In [ ]:
initialize_waveform = wg.Button(
    description='Initialize Waveform',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='This will initialize the waveform to zero volts in preparation for an ascending waveform',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
display(initialize_waveform)
initialize_waveform.on_click(initialize_bottom)

In [ ]:
rigol.query(':SOUR1:BURST:IDLE?')